In [1]:
print("hello")

hello


In [36]:
from datasets import load_dataset

# Load Dataset
dataset_name = "glue"
task_name = "sst2"
dataset = load_dataset(dataset_name, task_name, split="train")
validation_set = load_dataset('glue', 'sst2', split='validation')

Found cached dataset glue (/mnt/nlp4sd/hugging_face_cache/ysong/datasets/glue/sst2/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)
Found cached dataset glue (/mnt/nlp4sd/hugging_face_cache/ysong/datasets/glue/sst2/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)


In [37]:
def create_label_text(label):
    label_map = {
        0 : 'negative',
        1 : 'positive',
    }
    
    return {'label_text': label_map[label]}

In [38]:
# Convert digital labels to text labels
dataset = dataset.map(create_label_text, input_columns=['label'])
validation_set = validation_set.map(create_label_text, input_columns=['label'])

Loading cached processed dataset at /mnt/nlp4sd/hugging_face_cache/ysong/datasets/glue/sst2/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad/cache-ad792f34c7b5c010.arrow
Loading cached processed dataset at /mnt/nlp4sd/hugging_face_cache/ysong/datasets/glue/sst2/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad/cache-bb9884afcd56c7c4.arrow


In [5]:
dataset[0]

{'sentence': 'hide new secretions from the parental units ',
 'label': 0,
 'idx': 0,
 'label_text': 'negative'}

In [39]:
validation_set[0]

{'sentence': "it 's a charming and often affecting journey . ",
 'label': 1,
 'idx': 0,
 'label_text': 'positive'}

In [40]:
def convert_into_prompt_template(system_prompt, user_message, label_text=""):
    """
    Convert the dataset into the prompt template format as follows:
    <s>[INST] <<SYS>>
    {{ system_prompt }}
    <</SYS>>

    {{ user_message }} [/INST] 
    Sentiment: {{ label }} </s>      ## label is neccessary for training
    """
    text = f"<s>[INST] <<SYS>>\n{system_prompt}\n<</SYS>>\n\nSentence: {user_message} [/INST]\nSentiment: {label_text} </s>"
    return text

In [41]:
system_prompt = "You are a helpful, respectful and honest sentiment analysis assistant. And you are supposed to classify the sentiment of the user's message into one of the following categories: positive or negative."

### Simple test (Useless Right Now)

In [8]:
user_msg_2 = dataset[0]["sentence"]
label_2 = dataset[0]["label"]
converted1 = convert_into_prompt_template(system_prompt, user_msg_2, label_2)
print(converted1)

<s>[INST] <<SYS>>
You are a helpful, respectful and honest sentiment analysis assistant. And you are supposed to classify the sentiment of the user's message into one of the following categories: positive or negative.
<</SYS>>

hide new secretions from the parental units  [/INST]
Sentiment: 0 </s>


### Continue

In [42]:
def map_dataset(system_prompt, dataset):
    # Convert the dataset into the format required by the model
    def convert(sentence, label_text):
        converted_inputs = convert_into_prompt_template(system_prompt, sentence, label_text)
        return {'text': converted_inputs}

    return dataset.map(convert, input_columns=['sentence', 'label_text'], batched=False, remove_columns=['sentence', 'label', 'idx', 'label_text'])

In [43]:
new_dataset = map_dataset(system_prompt, dataset)
new_validation_set = map_dataset(system_prompt, validation_set)

Loading cached processed dataset at /mnt/nlp4sd/hugging_face_cache/ysong/datasets/glue/sst2/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad/cache-709cb218eec8d0aa.arrow


Map:   0%|          | 0/872 [00:00<?, ? examples/s]

In [44]:
new_dataset[0]

{'text': "<s>[INST] <<SYS>>\nYou are a helpful, respectful and honest sentiment analysis assistant. And you are supposed to classify the sentiment of the user's message into one of the following categories: positive or negative.\n<</SYS>>\n\nSentence: hide new secretions from the parental units  [/INST]\nSentiment: negative </s>"}

In [45]:
new_validation_set[0]

{'text': "<s>[INST] <<SYS>>\nYou are a helpful, respectful and honest sentiment analysis assistant. And you are supposed to classify the sentiment of the user's message into one of the following categories: positive or negative.\n<</SYS>>\n\nSentence: it 's a charming and often affecting journey .  [/INST]\nSentiment: positive </s>"}

### Merge datasets

In [46]:
from datasets import DatasetDict

dataset_to_upload = DatasetDict({
    'train': new_dataset,
    'validation': new_validation_set
})

In [47]:
dataset_to_upload

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 67349
    })
    validation: Dataset({
        features: ['text'],
        num_rows: 872
    })
})

### Upload dataset

In [49]:
dataset_to_upload.push_to_hub("OneFly7/llama2-sst2-finetuning")

Pushing split train to the Hub.


Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Pushing split validation to the Hub.


Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]